In [4]:
import os
import cv2
import shutil
import mediapipe as mp
import numpy as np

# Initialize Mediapipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Prepare Data
data_dir = './data2'
train_dir = os.path.join(data_dir, 'train')
input_folders = ['tricep pushdown']

# Create train directory if it doesn't exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

# Function for automatic brightness and contrast adjustment
def automatic_brightness_contrast(image, clip_hist_percent=1):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    hist_size = len(hist)
    accumulator = [float(hist[0])]
    for index in range(1, hist_size):
        accumulator.append(accumulator[index - 1] + float(hist[index]))
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum / 100.0) / 2.0
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1
    maximum_gray = hist_size - 1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha
    auto_result = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return auto_result

# Process images from input folders
for folder in input_folders:
    folder_path = os.path.join('.', folder)
    output_folder_path = os.path.join(train_dir, folder)

    # Create output subdirectory for the current folder if it doesn't exist
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        image = cv2.imread(image_path)
        if image is None:
            continue  # Skip files that are not images

        # Flip the image horizontally (for classification)
        flipped_image = cv2.flip(image, 1)

        # Save the original image to the training set
        shutil.copy(image_path, output_folder_path)

        ### Apply automatic brightness and contrast adjustment
        adjusted_image = automatic_brightness_contrast(flipped_image)

        #flipped_image_path = os.path.join(output_folder_path, f"flipped_{image_name}")
        #cv2.imwrite(flipped_image_path, flipped_image)
        
        ### Save the adjusted image to the training set
        adjusted_image_path = os.path.join(output_folder_path, f"adjusted_{image_name}")
        cv2.imwrite(adjusted_image_path, adjusted_image)

print("Image processing completed. Adjusted images saved in './data2/train'.")

C:\Users\ideapad GAMING\AppData\Local\Temp\ipykernel_4792\1659876271.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  accumulator = [float(hist[0])]
C:\Users\ideapad GAMING\AppData\Local\Temp\ipykernel_4792\1659876271.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  accumulator.append(accumulator[index - 1] + float(hist[index]))


Image processing completed. Adjusted images saved in './data2/train'.
